In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim


import sys
sys.path.append('../src')
from ds_utils import *
from torch_utils import *

In [3]:
#configuration
ROOT_DIR = '../../'
experiment_name = 'training_single_split'
learning_rate = 0.00001
epochs = 80
patience = 10
resnet_size = 34 # allowed sizes: 18,34,50,101,152
num_workers = 4
batch_size = 64
weighted_loss = True
#img_aug = None

In [4]:
from imgaug import augmenters as iaa

prob_aug = 0.5
sometimes = lambda augmentation: iaa.Sometimes(prob_aug, augmentation)
img_aug = iaa.Sequential([
    iaa.Fliplr(prob_aug),
    sometimes(iaa.Crop(percent=(0, 0.2))),
    sometimes(iaa.ChangeColorTemperature((1100, 10000))),

    sometimes(iaa.OneOf([
        iaa.GaussianBlur(sigma=(0, 2.0)),
        iaa.AddToHueAndSaturation((-10, 10))

    ]))

])


In [5]:
#load data into a dataframe containing the path and the category of each image
df = path2DataFrame('/home/jcejudo/training_data_3000')

In [6]:
df.head()

,file_path,category
0,/home/jcejudo/training_data_3000/building/[ph]...,building
1,/home/jcejudo/training_data_3000/building/[ph]...,building
2,/home/jcejudo/training_data_3000/building/[ph]...,building
3,/home/jcejudo/training_data_3000/building/[ph]...,building
4,/home/jcejudo/training_data_3000/building/[ph]...,building


In [7]:
# apply label encoding https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
X = df['file_path'].values
y = df['category'].values
y_encoded, class_index_dict = label_encoding(y)
n_classes = len(class_index_dict)

In [8]:
n_classes

20

In [9]:
class_index_dict

{0: 'archaeological_site',
 1: 'building',
 2: 'ceramics',
 3: 'clothing',
 4: 'costume_accessories',
 5: 'drawing',
 6: 'furniture',
 7: 'inscription',
 8: 'jewellery',
 9: 'map',
 10: 'painting',
 11: 'photograph',
 12: 'postcard',
 13: 'sculpture',
 14: 'specimen',
 15: 'tapestry',
 16: 'textile',
 17: 'toy',
 18: 'weaponry',
 19: 'woodwork'}

In [10]:
#split the data into train, validation and test set

#this function makes several splits
data_splits = make_train_val_test_splits(
    X,
    y_encoded,
    img_aug = img_aug,
    num_workers = num_workers,
    batch_size = batch_size,
    splits = 10,
)

#pick a single split
split = data_splits[0]

trainloader = split['trainloader']
valloader = split['valloader']
testloader = split['testloader']

print('size train: {}'.format(len(trainloader.dataset)))
print('size val: {}'.format(len(valloader.dataset)))
print('size test: {}'.format(len(testloader.dataset)))

size train: 37711
size val: 4191
size test: 4656


In [11]:
# GPU
device = torch.device('cuda:0')

#initialize model
model = ResNet(resnet_size,n_classes).to(device)
#set optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#set loss
if weighted_loss:
    #A weighted loss can be used for addressing the imbalance between categories
    #Categories with a small number of categories are assigned a higher loss so the model is force to correctly classify them 
    weights = get_class_weights(y_encoded,class_index_dict)
    loss_function = nn.CrossEntropyLoss(reduction ='sum',weight=torch.FloatTensor(weights).to(device))           
else:
    loss_function = nn.CrossEntropyLoss(reduction='sum')
    
#create directories for results
experiment_path = os.path.join(ROOT_DIR,experiment_name)
create_dir(experiment_path)
split_path = os.path.join(experiment_path,'split')
create_dir(split_path)


In [ ]:
model, history = train(
    model = model,
    loss_function = loss_function,
    optimizer = optimizer,
    trainloader = trainloader,
    valloader = valloader,
    device = device,
    saving_dir = split_path,
    epochs = epochs,
    patience = patience)


#save model data and training history
experiment = Experiment()
experiment.add('class_index_dict',class_index_dict)
experiment.add('model',model)
experiment.add('resnet_size',resnet_size)

for k,v in history.items():
    experiment.add(k,v)

experiment.save(split_path)

Training for 80 epochs 

[0] train loss: 73.927 validation loss: 69.024 acc: 0.518 f1: 0.400 precision: 0.441 recall: 0.476
[1] train loss: 68.660 validation loss: 67.383 acc: 0.576 f1: 0.457 precision: 0.455 recall: 0.528
[2] train loss: 67.425 validation loss: 66.614 acc: 0.627 f1: 0.509 precision: 0.501 recall: 0.571
[3] train loss: 66.469 validation loss: 66.068 acc: 0.665 f1: 0.541 precision: 0.559 recall: 0.601
[4] train loss: 65.662 validation loss: 65.105 acc: 0.714 f1: 0.595 precision: 0.576 recall: 0.639
[5] train loss: 64.988 validation loss: 64.804 acc: 0.729 f1: 0.607 precision: 0.584 recall: 0.651
[6] train loss: 64.528 validation loss: 64.439 acc: 0.759 f1: 0.651 precision: 0.647 recall: 0.675
[7] train loss: 64.063 validation loss: 64.148 acc: 0.772 f1: 0.666 precision: 0.662 recall: 0.682
[8] train loss: 63.709 validation loss: 63.864 acc: 0.783 f1: 0.679 precision: 0.673 recall: 0.695
[9] train loss: 63.510 validation loss: 63.887 acc: 0.788 f1: 0.682 precision: 0.672

In [ ]:

metrics_list = ['accuracy','f1','precision','recall','sensitivity','specificity']
max_epochs = 5
training_history_dict = read_train_history(experiment_path,metrics_list,max_epochs)

#plot loss on train and validation
plot_mean_std({metric:np.array(training_history_dict[metric]) for metric in ['loss_val','loss_train']},'',fontsize=20)  


In [ ]:
#plot metrics on validation data
plot_mean_std({metric:np.array(training_history_dict[metric]) for metric in metrics_list},'',fontsize=20,y_lim = 1.0)  


In [ ]:
# evaluate on test data
test_metrics, ground_truth_list, predictions_list,test_images_list = evaluate(model = model,
                                                                              dataloader = testloader,
                                                                              device = device,
                                                                              loss_function = loss_function)
for k in metrics_list:
    print(f'{k}_test: {test_metrics[k]}')

In [ ]:
#visualize confusion matrix
confusion_matrix = test_metrics['confusion_matrix']
labels = [class_index_dict[i] for i in range(confusion_matrix.shape[0])]
plot_conf_matrix(confusion_matrix,labels,font_scale=1)